<a href="https://colab.research.google.com/github/mariokart345/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [3]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [4]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [5]:
#Removing Outliers and restricting dataset for One Family Dwellings
df = df[df['BUILDING_CLASS_CATEGORY']=='01 ONE FAMILY DWELLINGS']
df.drop(columns='BUILDING_CLASS_CATEGORY',inplace=True)
df = df[(df['SALE_PRICE'] > 100000)&(df['SALE_PRICE'] < 2000000)]
#Coverting Column to datetime to then seperate January-March for training data whilst April as testing
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
train = df[df['SALE_DATE'] < '2019-03-31']
test = df[df['SALE_DATE'] >= '2019-03-31']

In [6]:
#Making feature and target vector dataframes
x_train,x_test = train.drop(columns=['SALE_PRICE', 'SALE_DATE','EASE-MENT']),test.drop(columns=['SALE_PRICE','SALE_DATE','EASE-MENT'])
y_train,y_test = train['SALE_PRICE'],test['SALE_PRICE']

In [ ]:
#import and Instantiate transformers
from category_encoders import OneHotEncoder
from sklearn.feature_selection import SelectKBest
encode = OneHotEncoder(use_cat_names=True)
SKBest = SelectKBest(k=30)
#Fiting training data and transforming test data(OneHotEncoder)
XT_train = encode.fit_transform(x_train)
XT_test = encode.transform(x_test)
#Fiting training data and transforming test data(SelectKBest)
XTT_train = SKBest.fit_transform(XT_train,y_train)
XTT_test = SKBest.transform(XT_test)

In [27]:
#BaseLine Before going into Regression
from sklearn.metrics import mean_absolute_error
y_pred = [y_train.mean()] * len(y_train)
Baseline_mae = mean_absolute_error(y_train,y_pred)
print(f'Baseline MAE:{Baseline_mae}')

Baseline MAE:214721.52773001452


In [48]:
#Instatinating Models
from sklearn.linear_model import LinearRegression, Ridge
LRModel = LinearRegression()
RModel = Ridge(normalize=True)
#
LRModel.fit(XTT_train,y_train)
RModel.fit(XT_train,y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
      random_state=None, solver='auto', tol=0.001)

In [54]:
from sklearn.metrics import mean_absolute_error
print(f'LRModel Train MAE:{mean_absolute_error(y_train,LRModel.predict(XTT_train))}\nLRModel Test MAE:{mean_absolute_error(y_test,LRModel.predict(XTT_test))}')
print(f'\nRModel Train MAE:{mean_absolute_error(y_train,RModel.predict(XT_train))}\nRModel Test MAE:{mean_absolute_error(y_test,RModel.predict(XT_test))}')

LRModel Train MAE:208912.08465436866
LRModel Test MAE:211634.35911190408

RModel Train MAE:79139.21244553318
RModel Test MAE:181751.18541401278
